### Bài toán

https://blog.luyencode.net/phan-loai-van-ban-tieng-viet/#bai-toan-phan-loai-van-ban

- Input: một đoạn văn bản
- Output: một trong các nhãn sau: 'thể thao', 'giáo dục', 'giải trí', 'kinh doanh', 'pháp luật', 'sức khỏe', 'số hóa', 'thế giới', 'thời sự', 'xe'



### Thu thập dữ liệu


Dữ liệu được lấy dừ https://github.com/duyvuleo/VNTC/tree/master/Data/10Topics/Ver1.1 với thông tin như sau:

***Train***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	5219 |
| Doi song	| DS |		3159 |
| Khoa hoc	| KH |		1820 |
| Kinh doanh	| KD |		2552 |
| Phap luat	| PL |		3868 |
| Suc khoe	| SK |		3384 |
| The gioi	| TG |		2898 |
| The thao	| TT |		5298 |
| Van hoa 	| VH |		3080 |
| Vi tinh		| VT |		2481 |

Total				33759

***Test***
|Topic |	Topic ID |	#files |
|--|--|--|
| Chinh tri Xa hoi	| XH |	7567 |
| Doi song	| DS |		2036 |
| Khoa hoc	| KH |		2096 |
| Kinh doanh	| KD |		5276 |
| Phap luat	| PL |		3788 |
| Suc khoe	| SK |		5417 |
| The gioi	| TG |		6716 |
| The thao	| TT |		6667 |
| Van hoa		| VH |		6250 |
| Vi tinh		| VT |		4560 |

Total				50373

Tên file được đặt cào từ:

+ DS_VNE_(...) : VnExpress news agency (http://vnexpress.net/)
+ DS_TT_(...):  Youth news agency (http://tuoitre.vn/)
+ DS_TN_(...): Thanh Nien news agency (http://thanhnien.vn/)
+ DS_NLD_(...): Nguoi Lao Dong news agency (http://nld.com.vn/)

File zip chứa toàn bộ file, tên mỗi file là Nhãn_Báo_ (STT).txt VD: XH_NLD_ (3675).txt

In [1]:
import zipfile

def extract_data_from_zip(zip_file_path):
    result = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()  # Lấy danh sách tên các file trong zip

        for file_name in file_list:
            if file_name.endswith('.txt'):  # Chỉ xử lý các file có đuôi .txt
                with zip_ref.open(file_name) as file:
                    data = file.read().decode('utf-16-le')  # Nó được mã hoá bằng utf-16-le
                    # nội dung file có dạng \ufeff<content>, đôi khi có dấu cách ở đầu và cuối
                    content = data[1:].strip()
                    label = file_name.split('/')[2].split('_')[:2]
                    result.append((content, label))
    return result

In [2]:
train_data = extract_data_from_zip('data/Train_Full.zip')
len(train_data)

33759

In [3]:
test_data = extract_data_from_zip('data/Test_Full.zip')
len(test_data)

50373

### Tiền xử lý dữ liệu

#### Xoá HTML

Dữ liệu được thu thập từ các website đôi khi vẫn còn sót lại các đoạn mã HTML. Các mã HTML code này là rác, chẳng những không có tác dụng cho việc phân loại mà còn làm kết quả phân loại văn bản bị kém đi. Do đó, cần phải loại bỏ các đoạn mã HTML này.

In [1]:
import re
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

txt = "<p class=\"par\">This is an example</p>"
remove_html(txt)

'This is an example'

#### Chuẩn hoá Tiếng Việt

- **Chuẩn hoá Unicode**: Hiện nay, có 2 loại mã Unicode được sử dụng phổ biến, Unicode tổ hợp và Unicode dựng sẵn. Hướng xử lý: Đưa về 1 chuẩn Unicode dựng sẵn (thằng này phổ biến hơn)
- **Chuẩn hoán cách bỏ dấu**: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý

In [ ]:
# -*- coding: utf-8 -*-

import re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)


"""
    Start section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
    Ví dụ: thủy = thuyr, tượng = tuwowngj
"""
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)


def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word


def vn_sentence_to_telex_type(sentence):
    """
    Chuyển câu tiếng việt có dấu về kiểu gõ telex.
    :param sentence:
    :return:
    """
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
"""

"""
    Start section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_t%E1%BA%AFc_%C4%91%E1%BA%B7t_d%E1%BA%A5u_thanh_trong_ch%E1%BB%AF_qu%E1%BB%91c_ng%E1%BB%AF
"""


def vietnameseWordNormalizer(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def vietnameseTextNormalizer(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = vietnameseWordNormalizer(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_tắc_đặt_dấu_thanh_trong_chữ_quốc_ngữ
"""
if __name__ == '__main__':
    print(vietnameseTextNormalizer('anh hoà, đang làm.. gì'))
    # f = open('/home/lap60313/data/corpus-full.txt', encoding='utf8')
    # sentence = f.readline()
    # current_line = 0
    # while sentence:
    #     current_line += 1
    #     if current_line % 1000 == 0:
    #         print('Current line', str(current_line))
    #     sentence = sentence.lower().strip()
    #     sentence = convertwindown1525toutf8(sentence)
    #     sentence = vietnameseTextNormalizer(sentence)
    #     with open('/home/lap60313/data/corpus-full.txt.out', 'a+', encoding='utf8') as fp:
    #         fp.write(sentence + "\n")
    #     sentence = f.readline()

#### Tách từ

File từ điển các từ và cụm từ được tổng hợp từ các nguồn sau:
- Viet74K.txt: https://github.com/undertheseanlp/underthesea/tree/main/underthesea/corpus/data
- words.txt: https://github.com/undertheseanlp/underthesea/tree/main/datasets/DI_Vietnamese-UVD/corpus/dictionary
- vi-vocab: https://github.com/vncorenlp/VnCoreNLP/tree/master/models/wordsegmenter
- Thư mục Words - Danh mục từ của wordnet: https://github.com/zeloru/vietnamese-wordnet/tree/master

Sau đó được xử lý để tạo thành 1 file từ điển duy nhất.

In [5]:
from collections import defaultdict
import unicodedata as ud
import re

def syllablize(sentence): # Tách âm tiết cho một câu tiếng Việt
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

# Tải từ trong vi-vocab.txt
with open('data/dic.txt', encoding='utf8') as f:
    vocab = f.read().split('\n')
# Xây dựng từ điển vocabs theo độ dài từ
vocabs = defaultdict(list)
for word in vocab:
    vocabs[len(word.split())].append(word)

print('Số lượng từ ghép và cụm từ trong vocab:', len(vocab))
print('Số bộ vocab phân theo độ dài:', len(vocabs))

def longest_matching(sentence, vocabs):
  words = syllablize(sentence) # tách âm tiết cho câu
  result = []
  i = len(words)-1 # index của từ hiện tại
  while i > -1: 
    word = '' 
    # tìm kiếm trong từ điển theo chiều dài của từ ưu tiên từ dài trước
    for j in range(i+1):
      ls_word = words[j:i+1]
      word = ' '.join(ls_word)
      # xem thử có trong từ điển không
      if word.lower() in vocabs.get(len(ls_word), []):
        i = j
        break
    result = [word] + result
    i-=1
  return result # return the final list

def tokenize_sentences(sentence):
    return ' '.join([x.replace(' ','_') for x in longest_matching(sentence, vocabs)])

tokenize_sentences('nhưng sự thực hiện vẫn còn chưa phù hợp')

Số lượng từ ghép và cụm từ trong vocab: 69335
Số bộ vocab phân theo độ dài: 19


'nhưng sự thực_hiện vẫn còn chưa phù_hợp'

#### Hoàn thiện tiền xử lý

In [ ]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = vietnameseTextNormalizer(document)
    # tách từ
    document = tokenize_sentences(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

#### Loại bỏ stopword

In [ ]:
with open('data/stopwords.txt', encoding='utf8') as f:
    stopword = f.read().replace(' ','_').split('\n')

stopword = set(stopword)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

### Xây dựng mô hình phân loại văn bản

#### Xây dựng tập dữ liệu huấn luyện và kiểm thử

#### Xây dựng mô hình phân loại

### Đánh giá mô hình phân loại văn bản

### Tài liệu tham khảo